In [ ]:
import pandas as pd

df = pd.read_csv('data/inf.csv', sep=',', index_col=0, parse_dates=True) # it's important to define index and parse_dates

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
ags_df = pd.read_json('data/ags.json').transpose()

def get_ags(lk_name):
    row = ags_df[ags_df['name'] == lk_name]
    return f'{row.index.values[0]}'

def get_pop(lk_name):
    row = ags_df[ags_df['name'] == lk_name]
    return row.population.values[0]

ags_df[ags_df['name'] == 'SK München']

In [ ]:
get_pop('SK München')

In [ ]:
def trend(x):
    if x < -0.05:
        return 'DOWN_STRONG'
    elif x < -0.01:
        return 'DOWN'
    elif x < 0.01:
        return 'STEADY'
    elif x < 0.05:
        return 'UP'
    else:
        return 'UP_STRONG'

def prepare_lk(lk_name):
    lk = df[get_ags(lk_name)].to_frame()
    lk = lk.rename(columns={f"{get_ags(lk_name)}": "inf_sum"})
    
    
    lk['dif'] = lk.inf_sum - lk.inf_sum.shift(1)
    lk['dif'] = lk['dif'].fillna(0)
    lk = lk.astype({'dif': 'int64'})

    lk['rolling_dif'] = lk.dif.rolling(7).mean()

    lk['inf_7_tage'] = lk.inf_sum  - lk.inf_sum.shift(7)
    lk['inf_7_tage'] = lk['inf_7_tage'].fillna(method='bfill')
    lk = lk.astype({'inf_7_tage': 'int64'})

    lk['inz'] = round(lk['inf_7_tage'] / (get_pop(lk_name) / 100_000),1)

    lk['inz_trend'] = round((lk['inz'] - lk.inz.shift(1)) / lk.inz,2)
    lk['inz_trend'] = lk.apply(lambda x: trend(x['inz_trend']), axis=1)
    return lk

def plot_lk_abs(lk_df):
    lk_df['dif'].plot.line(x='date', figsize=(18,8), color='silver', linewidth=0.8)
    lk_df['rolling_dif'].plot.line(x='date', figsize=(18,8), color='cornflowerblue', linewidth=7)

def plot_lk_inz(lk_df, color='cornflowerblue'):
    #lk_df['inf_7_tage'].plot.line(x='date', figsize=(18,8), color='silver', linewidth=0.8)
    lk_df['inz'].plot.line(x='date', figsize=(18,8), color=color, linewidth=7)

In [ ]:
muc = prepare_lk('SK München')
muc.tail(60)


In [ ]:
plot_lk_inz(prepare_lk('SK München'), color='green')
#plot_lk_inz(prepare_lk('SK Flensburg'),color='lightblue')
plot_lk_inz(prepare_lk('Berlin'),color='grey')
#plot_lk_inz(prepare_lk('LK Tirschenreuth'),color='red')


In [ ]:
muc_90 = prepare_lk('SK München').tail(90)

plot_lk_inz(muc_90, color='salmon')

In [ ]:
row = muc_90[muc_90['inz'] == min(muc_90['inz'])]
row

In [ ]:
muc_recent = prepare_lk('SK München').tail(50)

plot_lk_inz(muc_recent, color='tomato')

In [ ]:
muc_recent[muc_recent['inz'] == min(muc_recent['inz'])]

In [ ]:
muc_recent[muc_recent['inz'] == max(muc_recent['inz'])]

muc_recent.tail(30)
